In [54]:
import numpy as np
import pandas as pd

from pycaret.classification import *

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('user uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [55]:
text = pd.read_csv("text.csv")

In [56]:
text.head(10)

,Unnamed: 0,description_x,description_y,ticker_x,ticker_y,same_security
0,0,first trust dow jones internet,first trust dj internet idx,FDN,FDN,True
1,1,schwab intl large company index etf,schwab strategic tr fundamental intl large co ...,FNDF,FNDF,True
2,2,vanguard small cap index adm,vanguard small-cap index fund inst,VSMAX,VSCIX,False
3,3,duke energy corp new com new isin #us4 sedol #...,duke energy corp new com new isin #us26441c204...,DUK,DUK,True
4,4,visa inc class a,visa inc.,V,V,True
5,5,ford motor co new div: 0.600,ford motor co,F,F,True
6,6,united states steel corp,united sts stl cp (new),X,X,True
7,7,vanguard total international bond index etf,vanguard total intl bond index etf,BNDX,BNDX,True
8,8,schwab strategic tr us sml c,schwab us small cap etf,SCHA,SCHA,True
9,9,mfs value fd cl i,mfs value fund cl i,MEIIX,MEIIX,True


In [57]:
s = setup(data=text, target='same_security')


,Description,Value
0,Session id,2942
1,Target,same_security
2,Target type,Binary
3,Original data shape,"(2142, 6)"
4,Transformed data shape,"(2142, 6)"
5,Transformed train set shape,"(1499, 6)"
6,Transformed test set shape,"(643, 6)"
7,Numeric features,1
8,Categorical features,4
9,Preprocess,True


In [58]:
cm = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9253,0.9450,0.9840,0.9223,0.9521,0.7834,0.7930,0.3350
nb,Naive Bayes,0.9186,0.9486,0.9584,0.9355,0.9467,0.7745,0.7763,0.0940
rf,Random Forest Classifier,0.9146,0.9354,0.9885,0.9070,0.9458,0.7458,0.7631,0.2380
gbc,Gradient Boosting Classifier,0.9133,0.9108,0.9849,0.9082,0.9449,0.7433,0.7586,0.2160
ridge,Ridge Classifier,0.9106,0.9420,0.9832,0.9064,0.9431,0.7361,0.7507,0.1130
ada,Ada Boost Classifier,0.9093,0.9074,0.9832,0.9047,0.9423,0.7321,0.7468,0.1730
lda,Linear Discriminant Analysis,0.9079,0.8878,0.9734,0.9109,0.9410,0.7326,0.7424,0.0800
lightgbm,Light Gradient Boosting Machine,0.9079,0.9141,0.9849,0.9020,0.9416,0.7262,0.7432,0.3200
dt,Decision Tree Classifier,0.9073,0.8267,0.9858,0.9007,0.9413,0.7231,0.7413,0.1370
xgboost,Extreme Gradient Boosting,0.9053,0.9139,0.9477,0.9283,0.9366,0.7456,0.7527,0.1630


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [64]:
r = pull()

In [65]:
r

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec),TOPSIS SCORE,RANK
et,Extra Trees Classifier,0.9253,0.9450,0.9840,0.9223,0.9521,0.7834,0.7930,0.335,0.718584,10.0
nb,Naive Bayes,0.9186,0.9486,0.9584,0.9355,0.9467,0.7745,0.7763,0.094,0.887472,4.0
rf,Random Forest Classifier,0.9146,0.9354,0.9885,0.9070,0.9458,0.7458,0.7631,0.238,0.795977,9.0
gbc,Gradient Boosting Classifier,0.9133,0.9108,0.9849,0.9082,0.9449,0.7433,0.7586,0.216,0.812677,8.0
ridge,Ridge Classifier,0.9106,0.9420,0.9832,0.9064,0.9431,0.7361,0.7507,0.113,0.911866,1.0
ada,Ada Boost Classifier,0.9093,0.9074,0.9832,0.9047,0.9423,0.7321,0.7468,0.173,0.849929,6.0
lda,Linear Discriminant Analysis,0.9079,0.8878,0.9734,0.9109,0.9410,0.7326,0.7424,0.080,0.895416,3.0
lightgbm,Light Gradient Boosting Machine,0.9079,0.9141,0.9849,0.9020,0.9416,0.7262,0.7432,0.320,0.713611,11.0
dt,Decision Tree Classifier,0.9073,0.8267,0.9858,0.9007,0.9413,0.7231,0.7413,0.137,0.853418,5.0
xgboost,Extreme Gradient Boosting,0.9053,0.9139,0.9477,0.9283,0.9366,0.7456,0.7527,0.163,0.815563,7.0


In [61]:
from sklearn.preprocessing import MinMaxScaler

m = r.iloc[:, 1:].copy()  # metrics

scaler = MinMaxScaler()

# normalised metrics
n = pd.DataFrame(scaler.fit_transform(m), columns=m.columns)

weights = np.array([1, 1, 1, 1, 1, 1, 1, 1])
impacts = np.array(['+', '+', '+', '+', '+', '+', '+', '-'])

w = n * weights # weighted metrics

ideal_b = np.where(impacts == '+', w.max(), w.min())
ideal_w = np.where(impacts == '+', w.min(), w.max())

# euclidean distance
dist_b = np.linalg.norm(w - ideal_b, axis=1)
dist_w = np.linalg.norm(w - ideal_w, axis=1)

# TOPSIS SCORE
score = dist_w / (dist_b + dist_w)

# Rank Models
r["TOPSIS SCORE"] = score
r["RANK"] = r["TOPSIS SCORE"].rank(ascending=False)

In [62]:
r.sort_values(by="RANK")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec),TOPSIS SCORE,RANK
ridge,Ridge Classifier,0.9106,0.9420,0.9832,0.9064,0.9431,0.7361,0.7507,0.113,0.911866,1.0
lr,Logistic Regression,0.9013,0.9498,0.9867,0.8936,0.9378,0.7017,0.7234,0.105,0.897454,2.0
lda,Linear Discriminant Analysis,0.9079,0.8878,0.9734,0.9109,0.9410,0.7326,0.7424,0.080,0.895416,3.0
nb,Naive Bayes,0.9186,0.9486,0.9584,0.9355,0.9467,0.7745,0.7763,0.094,0.887472,4.0
dt,Decision Tree Classifier,0.9073,0.8267,0.9858,0.9007,0.9413,0.7231,0.7413,0.137,0.853418,5.0
ada,Ada Boost Classifier,0.9093,0.9074,0.9832,0.9047,0.9423,0.7321,0.7468,0.173,0.849929,6.0
xgboost,Extreme Gradient Boosting,0.9053,0.9139,0.9477,0.9283,0.9366,0.7456,0.7527,0.163,0.815563,7.0
gbc,Gradient Boosting Classifier,0.9133,0.9108,0.9849,0.9082,0.9449,0.7433,0.7586,0.216,0.812677,8.0
rf,Random Forest Classifier,0.9146,0.9354,0.9885,0.9070,0.9458,0.7458,0.7631,0.238,0.795977,9.0
et,Extra Trees Classifier,0.9253,0.9450,0.9840,0.9223,0.9521,0.7834,0.7930,0.335,0.718584,10.0
